In [7]:
# -------------------------------------------------- --------------------------------------- #
# STEP02_v1
    # A. Obtaining the CSD
    # B. Obtaining the centers of mass
    # C. Obtaining the trajectories of the centers of mass
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Custom made functions
# ----------------------------------------------------------------------------------------- #
# Path to the folder where the custom functions are storage
PATHMODULE = joinpath( homedir( ), "Dropbox", "CodigoGithub" ); #                           *
push!( LOAD_PATH, PATHMODULE );
using STEP02_v1
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Julia's native packages
# ----------------------------------------------------------------------------------------- #
using JLD2
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Searching for directories and files to use
# ----------------------------------------------------------------------------------------- #
PATHSTART = joinpath( homedir( ), "Desktop" ); #                                            *
PATHSINFO, _ = FindDirsFiles( PATHSTART, "Info" );
for i in 1:length( PATHSINFO )
    f = PATHSINFO[ i ];
    println( string( "$i - $f" ) );
end
Infos = length( PATHSINFO );
# ----------------------------------------------------------------------------------------- #
# Select a file to work with from now on.
# ----------------------------------------------------------------------------------------- #
info = 0;
while info > Infos || info < 1
    print( "Enter the file number: [ between 1 and $Infos ]" );
    info = parse( Int, readline( ) );
end
PATHINFO = PATHSINFO[ info ];
println( "\nPath selected: $PATHINFO" ); 
# ----------------------------------------------------------------------------------------- #

1 - /home/isabel/Desktop/Data/060915s01/Info
2 - /home/isabel/Desktop/Data/700_3-3/Info
3 - /home/isabel/Desktop/Data/lesion_parakarel_140116/Info
4 - /home/isabel/Desktop/GarbageCollector/lesion_parakarel_140116/Info
Enter the file number: [ between 1 and 4 ]stdin> 3

Path selected: /home/isabel/Desktop/Data/lesion_parakarel_140116/Info


In [8]:
# ----------------------------------------------------------------------------------------- #
# Generating the new folders and files
# ----------------------------------------------------------------------------------------- #
PATHMAIN = dirname( PATHINFO );
PATHFIGURES = joinpath( PATHMAIN, "Figures" );
PATHVOLTAGE = joinpath( PATHMAIN, "STEP01" );
PATHSTEP02 = joinpath( PATHMAIN, "STEP02" ); mkpath( PATHSTEP02 );
PATHCMS = joinpath( PATHMAIN, "CMS" ); mkpath( PATHCMS );
FILEPARAMETERS = joinpath( PATHINFO, "Parameters.jld2" );
#
FILEVARIABLES = joinpath( PATHINFO, "Variables.jld2" );
FILESVOLTAGE = SearchDir( PATHVOLTAGE, ".jld2" );
FILEACD = joinpath( PATHINFO, "ACD.jld2" );
# ----------------------------------------------------------------------------------------- #
# Extracting the variables contained in the brw file
# ----------------------------------------------------------------------------------------- #
Variables = LoadDict( FILEVARIABLES );
Parameters = LoadDict( FILEPARAMETERS );
ACD = LoadDict( FILEACD );
# ----------------------------------------------------------------------------------------- #

In [9]:
# ----------------------------------------------------------------------------------------- #
# Constants
# ----------------------------------------------------------------------------------------- #
#= 
    All of the following values fall into the * category, 
    meaning that they can or should be modified
=#
# ----------------------------------------------------------------------------------------- #
N = length( FILESVOLTAGE );
n0s = length( string( N ) );
σ = 3; # standar deviation for the gaussian kernel
# Minimum number of channels involved to consider the formation of a well or stream source 
minchannels = 3; 
tol_dist = 2; # Tolerance distance for mass center displacement
tol_time = 3; # Tolerance time for mass center displacement
#= Minimum time ( in frames ) necessary to consider that a center of mass traverses a 
trajectory =#
Tmin = 3; 
#= Minimum weight (in arbitrary units) required for a center of mass to be considered 
significant for trajectory analysis =# 
min_weight = 50; 
# ----------------------------------------------------------------------------------------- #

In [ ]:
# ----------------------------------------------------------------------------------------- #
# A. Loading the repaired voltage file, and obtaining the CSD
# ----------------------------------------------------------------------------------------- #
for n in 1:N
    # ------------------------------------------------------------------------------------- #
    # A. Obtaining the CSD
    # ------------------------------------------------------------------------------------- #
    BINNAME = joinpath( PATHVOLTAGE, string( "BIN", lpad( n, n0s, "0" ), ".jld2" ) );
    BINRAW = Float64.( LoadDict( BINNAME ) );
    nChs, nFrs = size( BINRAW );
    BINCSD = CSDA( BINRAW, σ );
    jldsave( replace( BINNAME, "STEP01" => "STEP02" ); Data = Float16.( BINCSD ) );
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    nc, nc, nFrs = size( BINCSD );
    # ------------------------------------------------------------------------------------- #

    # ----------------------------------------------------------------------------------------- #
    # Calculates the threshold for differentiating current generators
    # ----------------------------------------------------------------------------------------- #
    thr = DonohoMatrix3D( BINCSD );
    ϵ = thr .* SigmaData( BINCSD[ 1, 1, : ] );
    # ------------------------------------------------------------------------------------- #
    # B. Obtain the Centers of Mass
    # ----------------------------------------------------------------------------------------- #
    CMN, CMP = GetCentersOfMass( BINCSD, minchannels, ϵ );
    # ------------------------------------------------------------------------------------- #
    
    # ------------------------------------------------------------------------------------- #
    # C. Obtaining the trajectories of the centers of mass
    # ------------------------------------------------------------------------------------- #
    TP = Trajectories( CMP, tol_dist, tol_time, Tmin, min_weight );
    Starts, Stops = StartStop( TP );
    TPf = FixingGaps( TP, Starts, Stops );

    TN = Trajectories( CMN, tol_dist, tol_time, Tmin, min_weight );
    Starts, Stops = StartStop( TN );
    TNf = FixingGaps( TN, Starts, Stops );
    # ----------------------------------------------------------------------------------------- #
    Results = Dict(
        "CMN" => CMN,
        "CMP" => CMP,
        "TP" => TPf,
        "TN" => TNf
    );
    CMNAME = joinpath( PATHCMS, string( "BIN", lpad( n, n0s, "0" ), ".jld2" ) ); 
    jldsave( CMNAME; Data = Results );
    # ------------------------------------------------------------------------------------- #
end
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Saving the parameters for step-02
# ----------------------------------------------------------------------------------------- #
NewParameters = Dict(
    "sigma" => σ,
    "tol_dist" => tol_dist,
    "tol_time" => tol_time,
    "Tmin" => Tmin,
    "min_weight" => min_weight,
    "minchs_cm" => minchannels
);
Parameters = merge( Parameters, NewParameters );
jldsave( FILEPARAMETERS; Data = Parameters );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# end of the step-02
# ----------------------------------------------------------------------------------------- #